<a href="https://colab.research.google.com/github/omarcevi/ML-Projects/blob/main/TF_Distributed_training_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Download the Cassava dataset from TensorFlow
data, info = tfds.load(name='cassava', as_supervised=True, with_info=True)
NUM_CLASSES = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/5656 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/1885 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteWDG3TP/cassava-test.tfrecord*...:   0%|          |…

Generating validation examples...:   0%|          | 0/1889 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/cassava/0.1.0.incompleteWDG3TP/cassava-validation.tfrecord*...:   0%|     …

Dataset cassava downloaded and prepared to /root/tensorflow_datasets/cassava/0.1.0. Subsequent calls will reuse this data.


In [3]:
# Add a preprocess_data function to scale the image
def preprocess_data(image, label):
	image = tf.image.resize(image, (300, 300))
	return tf.cast(image, tf.float32) / 255., label

In [4]:
# Define the model
def create_model():
	base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False)
	x = base_model.output
	x = tf.keras.layers.GlobalAveragePooling2D()(x)
	x = tf.keras.layers.Dense(1016, activation='relu')(x)
	predictions = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(x)
	model = tf.keras.Model(inputs=base_model.input, outputs=predictions)
	return model

In [5]:
# Create the strategy object
strategy = tf.distribute.MirroredStrategy()

# Create model variables within the strategy scope
with strategy.scope():
	model = create_model()
	model.compile(
		loss='sparse_categorical_crossentropy',
		optimizer=tf.keras.optimizers.Adam(0.0001),
		metrics=['accuracy'])

94765736/94765736 [==============================] - 1s 0us/step


In [6]:
# Change the batch size
batch_size = 64 * strategy.num_replicas_in_sync

# Map, shuffle, and prefetch data
train_data = data['train'].map(preprocess_data)
train_data = train_data.shuffle(1000)
train_data = train_data.batch(batch_size)
train_data = train_data.prefetch(tf.data.experimental.AUTOTUNE)

model.fit(train_data, epochs=5)

Epoch 1/5
89/89 [==============================] - 173s 1s/step - loss: 0.6311 - accuracy: 0.7751
Epoch 2/5
89/89 [==============================] - 109s 1s/step - loss: 0.1813 - accuracy: 0.9429
Epoch 3/5
89/89 [==============================] - 110s 1s/step - loss: 0.0598 - accuracy: 0.9818
Epoch 4/5
89/89 [==============================] - 110s 1s/step - loss: 0.0418 - accuracy: 0.9878
Epoch 5/5
89/89 [==============================] - 111s 1s/step - loss: 0.0393 - accuracy: 0.9885
